In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import datetime as dt

# 시각화 관련
import plotly_express as px
import plotly.figure_factory as ff
import cufflinks as cf
cf.go_offline(connected=True)
## cf.getThemes()
cf.set_config_file(theme='polar')
import plotly.graph_objects as go

from matplotlib.colors import LinearSegmentedColormap
from matplotlib.ticker import FuncFormatter
import matplotlib.ticker as mticker
from matplotlib.ticker import LogLocator
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('whitegrid')
## plt.style.use("fivethirtyeight")
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams["figure.figsize"] = (10,6)


## 퀀트함수
from quant_functions import anal_funcs, data_funcs
import myfuncs
from fredapi import Fred
fred = Fred(api_key='cfb4f49f5c1a9396f671b8049d992e56')

In [2]:
## 색 설정
color_basic = "#323232" # 기본색(글자색)
color_around = "#6E6E6E" # 회색 주변색
color_around2 = "#E0E0E0" # 밝은 회색 주변색

color_norm = "#F23557" # 강조색(진홍색)
color_norm2 = "#A60A28" # 강조색 어두움(적갈색)
color_norm3 = "#F9A5B5" # 강조색 밝음(연분홍색)

color_cont = "#22b2da" # 대비색(청록색)
color_cont2 = "#146A82" # 대비색 어두움(진청록색)
color_cont3 = "#76D1EA" # 대비색 밝음(밝은 청록색)

color_sub = "#FFA305" # 부강조색(노란색)
color_sub2 = "#9C6400" # 부강조색 어두움(갈색)
color_sub3 = "#FFCB6D" # 부강조색 밝음(연한 노란색)

## figure size 설정
figsize=(12, 8)

In [3]:
# 블루 - 퍼플 테마
colors = [color_cont, color_cont2, color_cont3]
# LinearSegmentedColormap 생성
custom_cmap = LinearSegmentedColormap.from_list("custom_cmap", colors, N=256)

#### Data

In [4]:
dm_path = "C:/Users/sdjhs/git/data/DM"
## gold = pd.read_csv(f"{dm_path}/gold.csv", encoding='utf-8').set_index('Date')[['Close']].rename(columns={'Close':'Gold'})
gold = pd.read_csv(f"gold.csv", encoding='utf-8').set_index('Date')[['Close']].rename(columns={'Close':'Gold'})
gold.index = pd.to_datetime(gold.index)

df_ori = data_funcs.get_merged_df('^IXIC', '^GSPC', 'SCHD', 'GC=F', 'TLT', '^DJI')
DGS20 = fred.get_series('DGS20')
DGS10 = fred.get_series('DGS10')
df_ori = df_ori.fillna(method = 'pad')
df_ori['Gold'] = gold['Gold']
df_ori['DGS10'] = DGS10
df_ori['DGS20'] = DGS20

df_ori = df_ori[['SCHD', '^GSPC', '^DJI', '^IXIC', 'TLT', 'DGS10', 'DGS20', 'Gold', 'GC=F']]
df_ori.columns = ['SCHD', 'S&P500', 'DowJones', 'NASDAQ', 'TLT', 'DGS10', 'DGS20', 'Gold', 'GC=F']

[*********************100%%**********************]  6 of 6 completed


In [5]:
## Imputation
df_imputed = myfuncs.imputation(df_ori, 'S&P500', 'DowJones')
df_imputed = myfuncs.imputation(df_imputed, 'DowJones', 'SCHD')
df_imputed = myfuncs.imputation(df_imputed, 'DGS10', 'DGS20')
df_imputed = myfuncs.imputation(df_imputed, 'DGS20', 'TLT')
df_imputed = myfuncs.imputation(df_imputed, 'GC=F', 'Gold')
dataset = df_imputed['1970':][['SCHD', 'S&P500', 'TLT', 'Gold']]
dataset = dataset.fillna(method='pad')
dataset['Gold'][:gold.index[0]] = np.nan
dataset['Gold']['1975-01-02'] = 1
dataset['Gold']['1975-01-03'] = 1

0.919059677334056
0.8917614058452233
0.7565978823514551
-0.41908986077307586
0.9953455805075565


In [6]:
shock_dict = {"세계전충격_t0":'1948-11-01', "세계전충격_t1":'1949-10-28',
              "한국전충격_t0":'1953-07-01', "한국전충격_t1":'1954-05-28',
              "아이젠하워리세션_t0":'1957-08-01', "아이젠하워리세션_t1":'1958-04-28',
              "연착륙_t0":'1960-04-01', "연착륙_t1":'1961-02-28',
              "달러위기_t0":'1969-12-01', "달러위기_t1":'1970-10-29',
              "1차오일쇼크_t0":'1973-11-05', "1차오일쇼크_t1":'1975-03-03',
              "2차오일쇼크_t0":'1980-01-02', "2차오일쇼크_t1":'1980-07-02',
              "물가충격_t0":'1981-07-01', "물가충격_t1":'1982-11-03',
              "걸프전_t0":'1990-07-05', "걸프전_t1":'1991-03-01',
              "닷컴버블_t0":'2001-03-02', "닷컴버블_t1":'2001-11-02',
              "금융위기_t0":'2007-12-04', "금융위기_t1":'2009-06-01',
              "코로나_t0":'2020-01-31', "코로나_t1":'2020-04-03'}

In [7]:
assets = ['S&P500', 'TLT', 'Gold']

In [8]:
df = dataset[assets].copy()
df_1_0 = df[:'1984-12-31']
df_1_0 = df_1_0/df_1_0.iloc[0] * 100
## 1975년 이전 과거 금시세 참고
## https://www.macrotrends.net/1333/historical-gold-prices-100-year-chart

df = dataset[assets]['1975-01-02':].copy()
df = df/df.iloc[0] * 100
df_1 = df[:'1984-12-31']
df_1 = df_1/df_1.iloc[0] * 100
df_2 = df['1985-1-1':'1994-12-31']
df_2 = df_2/df_2.iloc[0] * 100
df_3 = df['1995-1-1':'2004-12-31']
df_3 = df_3/df_3.iloc[0] * 100
df_4 = df['2005-1-1':'2014-12-31']
df_4 = df_4/df_4.iloc[0] * 100
df_5 = df['2015-1-1':]
df_5 = df_5/df_5.iloc[0] * 100

In [9]:
#### Anualized Return

In [10]:
## AGR 그래프
def agr_plot(df):
    data = df.copy()
    ## 연간 수익률 계산
    data = myfuncs.calc_agr(data)
    ## 채권수익률 데이터 추가
    temp = df_imputed[['DGS10']].fillna(method='pad').copy()
    temp['year'] = temp.index.year
    temp = temp.groupby('year')[['DGS10']].mean()
    data['GS10'] = temp['DGS10'].copy()
    data = data[['GS10', 'S&P500']].rename(columns={'S&P500':'return'}).dropna()
    data['return'] = data['return'] * 100
    data.index = pd.to_datetime(data.index, format='%Y')

    fig, ax1 = plt.subplots(figsize=figsize)

    # GS10 데이터만 그리기
    ax1.plot(data.index, data['GS10'], color=color_cont3, label="GS10")
    ax1.set_xlabel("")
    ax1.set_ylabel("Yield", fontsize=14, fontweight="bold", color=color_around)
    ax1.set_ylim([-20, 20])
    ax1.set_xlim([min(data.index) - dt.timedelta(days=200),
            max(data.index) + dt.timedelta(days=200)])

    plt.gca().yaxis.set_major_formatter(FuncFormatter(myfuncs.percent_formatter))
    plt.gca().tick_params(axis="y", pad=1)  # Y축 눈금 패딩 조정
    plt.grid(color=color_around2, linestyle="--", linewidth=0.7, alpha=0.7)

    # Return 데이터를 bar로 그리기
    ax2 = ax1.twinx()
    performance_plus = data['return'][data['return'] >= 0]
    ax2.bar(performance_plus.index, performance_plus,
            color=color_norm, alpha=1, width=150, label="Growth(positive)")
    performance_minus = data['return'][data['return'] < 0]
    ax2.bar(performance_minus.index, performance_minus,
            color=color_cont, alpha=1, width=150, label="Growth(negative)")
    ax2.axhline(0, color=color_around, linestyle='--', linewidth=2)
    ax2.set_ylabel("Growth", fontsize=14, fontweight="bold", color=color_around)
    ax2.set_ylim([-60, 100])

    plt.gca().yaxis.set_major_formatter(FuncFormatter(myfuncs.percent_formatter))
    plt.gca().tick_params(axis="y", pad=1)  # Y축 눈금 패딩 조정
    plt.grid(color=color_around2, linestyle="--", linewidth=0.7, alpha=0.7)

    plt.title(f"Annual Growth Rate ({data.index.year[0]}~{data.index.year[-1]})", fontsize=22, fontweight='bold', color=color_basic)
    handler1, label1 = ax1.get_legend_handles_labels()
    handler2, label2 = ax2.get_legend_handles_labels()
    ax2.legend(handler1 + handler2, label1 + label2, loc=2, borderaxespad=0.)

    ## 경기침체
    ax1.axvspan(shock_dict['1차오일쇼크_t0'], shock_dict['1차오일쇼크_t1'], facecolor=color_around, alpha=0.45)
    ax1.axvspan(shock_dict['2차오일쇼크_t0'], shock_dict['2차오일쇼크_t1'], facecolor=color_around, alpha=0.45)
    ax1.axvspan(shock_dict['물가충격_t0'], shock_dict['물가충격_t1'], facecolor=color_around, alpha=0.45)
    ax1.axvspan(shock_dict['걸프전_t0'], shock_dict['걸프전_t1'], facecolor=color_around, alpha=0.45)
    ax1.axvspan(shock_dict['닷컴버블_t0'], shock_dict['닷컴버블_t1'], facecolor=color_around, alpha=0.45)
    ax1.axvspan(shock_dict['금융위기_t0'], shock_dict['금융위기_t1'], facecolor=color_around, alpha=0.45)
    ax1.axvspan(shock_dict['코로나_t0'], shock_dict['코로나_t1'], facecolor=color_around, alpha=0.45)

    plt.tight_layout()
    plt.show()

In [13]:
## agr_plot(df_1)

In [14]:
#### Trend

In [15]:
## 트랜드 그래프
def trend_plot(df):
    data = df[assets].copy()
    _df = data.reset_index()
    _df = _df.melt(id_vars='Date', value_vars=_df.columns, var_name='Ticker', value_name='Value')

    colors = [color_norm, color_cont, color_sub]
    lineplot = sns.lineplot(data=_df, x='Date', y='Value', hue='Ticker', palette=colors, linestyle='-', linewidth=1)
    plt.title(f'Trends of {", ".join(assets)} ({data.index[0].year}~{data.index[-1].year})', fontsize=22, fontweight='bold', color=color_basic)
    plt.ylabel(f"{data.index[0].year}Y=100", fontsize=14, labelpad=-40, fontweight="bold", color=color_around, loc="top", rotation=0)
    plt.xlabel("")
    plt.xticks(fontsize=12, color=color_around)
    plt.yticks(fontsize=12, color=color_around)

    # 눈금과 축 간격 줄이기
    plt.gca().tick_params(axis="y", pad=1)  # Y축 눈금 패딩 조정

    # 그리드 추가
    plt.grid(color=color_around2, linestyle="--", linewidth=0.7, alpha=0.7)

    x_min, x_max = data.index.min(), data.index.max()
    # 강조할 기간 목록
    highlight_periods = [
        ('1980-01-02', '1980-07-02'),
        ('1981-07-01', '1982-11-03'),
        ('1990-07-05', '1991-03-01'),
        ('2001-03-02', '2001-11-02'),
        ('2007-12-04', '2009-06-01'),
        ('2020-01-31', '2020-04-03')
    ]

    for start, end in highlight_periods:
        # datetime으로 변환
        start_date = pd.to_datetime(start)
        end_date = pd.to_datetime(end)
        
        # x축 범위와 비교하여 범위를 조정
        if start_date > x_max or end_date < x_min:
            continue  # x축 범위를 벗어나면 건너뜀
        adjusted_start = max(start_date, x_min)
        adjusted_end = min(end_date, x_max)
        plt.axvspan(adjusted_start, adjusted_end, facecolor=color_around, alpha=0.45)

    # 그래프 표시
    plt.tight_layout()
    plt.show()

In [17]:
## trend_plot(df_1)

In [18]:
#### MDD

In [23]:
## 전고점대비 최대낙폭(MDD)
def mdd_plot(df):
    mdd_df, hrr_df = anal_funcs.get_MDD_report(df_1, 'a')
    hrr_df = hrr_df*100

    data = hrr_df.copy()

    colors = [color_norm, color_cont, color_sub]  # 각 그래프에 사용할 색상
    colors_edge = [color_norm2, color_cont2, color_sub2]  # 각 그래프에 사용할 색상
    # 서브플롯 생성
    fig, axes = plt.subplots(1, len(assets), figsize=(12, 4), sharey=True)  # 가로 방향 서브플롯

    for i, asset in enumerate(assets):
        # 데이터 전처리
        _df = data[[asset]].dropna().reset_index()
        _df = _df.melt(id_vars='Date', value_vars=_df.columns, var_name='Ticker', value_name='Value')

        # 현재 서브플롯 선택
        ax = axes[i]
        
        # 라인 플롯
        sns.lineplot(data=_df, x='Date', y='Value', hue='Ticker', palette=[colors[i]], linestyle='-', linewidth=1, ax=ax)
        
        # 제목 및 축 설정
        ax.set_title(f'Drawdown of {asset} ({data.index[0].year}~{data.index[-1].year})', fontsize=14, fontweight='bold', color=color_basic)
        ax.set_ylabel("MDD", fontsize=12, labelpad=-20, fontweight="bold", color=color_around, rotation=0, loc="top")
        ax.set_xlabel("")
        ax.tick_params(axis="x", labelsize=10, colors=color_around)
        ax.tick_params(axis="y", labelsize=10, colors=color_around)
        
        # y축에 퍼센트 포맷 적용
        ax.yaxis.set_major_formatter(FuncFormatter(myfuncs.percent_formatter))
        
        # 그리드 추가
        ax.grid(color=color_around2, linestyle="--", linewidth=0.7, alpha=0.7)
        
        # mdd 지점
        ax.scatter(hrr_df[asset].idxmin(), hrr_df[asset].min(), color=colors_edge[i], s=50, zorder=5)
        print(hrr_df[asset].idxmin(), hrr_df[asset].min())
        
        # 강조 영역 추가
        highlight_periods = [
            ('1980-01-02', '1980-07-02'),
            ('1981-07-01', '1982-11-03'),
            ('1990-07-05', '1991-03-01'),
            ('2001-03-02', '2001-11-02'),
            ('2007-12-04', '2009-06-01'),
            ('2020-01-31', '2020-04-03')
        ]
        for start, end in highlight_periods:
            start_date = pd.to_datetime(start)
            end_date = pd.to_datetime(end)
            if start_date > _df['Date'].max() or end_date < _df['Date'].min():
                continue
            adjusted_start = max(start_date, _df['Date'].min())
            adjusted_end = min(end_date, _df['Date'].max())
            ax.axvspan(adjusted_start, adjusted_end, facecolor=color_around, alpha=0.45)

    # 레이아웃 조정 및 표시
    plt.tight_layout()
    plt.show()

In [22]:
## mdd_plot(df_1)

In [ ]:
#### Rolling-Return Trend

In [ ]:
#### Rolling-Return Valiation